In [22]:
#Importing pandas and json to open the datasets and perform the task required
import pandas as pd
import json

In [23]:
#files/dataset are present in drive folder so mounting drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
#converting the excel to dataframes
CRdf = pd.read_excel("/content/drive/My Drive/kelp/CreditRating.xlsx")
SCdf= pd.read_excel("/content/drive/My Drive/kelp/ScreenerCINMapping.xlsx")



In [25]:
#creating mapping of cin to screener link in the form of dictionary
cinLink = dict(zip(SCdf['CIN'], SCdf['Screener Link']))

In [26]:
#it is a function to extract screener link by providing CIN
def extract_link(cin_number):
    return cinLink.get(cin_number, "")

In [27]:
#applying function
CRdf['Link'] = CRdf['CIN'].apply(extract_link)

In [28]:
#viewing head of out credit rating dataframe
#CRdf.head()

In [29]:


# as rating deatils column has json string format,for getting desired output there is need
#to extract different values from each json string for respective CIN number
CIN = CRdf['CIN']
Rating_Details = CRdf['Rating Details']
Link = CRdf['Link']

#empty list to store json data when collected for each row
data = []

# Iterating through each row
for cin, rating_details, link in zip(CIN, Rating_Details, Link):
    # while processing there were errors for missing value and type of values hence try and except block is applied
    if pd.isna(rating_details):
        print(f"Rating details missing for row {cin}")
        continue

    try:
        #JSON string to dictionary
        rating_details = json.loads(rating_details)

        #extracting our desired values for new excel output from each key value pair in dictionary
        for detail in rating_details:

            date = detail.get('date', '')
            agency = detail.get('agency', '')
            amount = detail.get('amount', '')
            rating = detail.get('rating', '')
            instrument = detail.get('instrument', '')
            rating_status = detail.get('ratingStatus', '')
            rating_grade = detail.get('ratingGrade', '')

            # Appending data to the empty list
            data.append([cin, date, agency, amount, instrument, rating_status, rating_grade, link])
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in row {cin}: {e}")

# Creating DataFrame from the data we obtained and assigning required column names to dataframe
CRtoLink_df = pd.DataFrame(data, columns=['CIN number', 'Date of Rating', 'Rating Agency', 'Amount (Mn)',
                                        'Rating Instrument', 'Rating Status', 'Rating Grade', 'Link'])


Error decoding JSON in row L24200MH1989PLC051736: Expecting property name enclosed in double quotes: line 1 column 32768 (char 32767)
Error decoding JSON in row L85110KA1930PLC000124: Unterminated string starting at: line 1 column 32757 (char 32756)
Error decoding JSON in row L24121MH1979PLC021360: Unterminated string starting at: line 1 column 32759 (char 32758)
Error decoding JSON in row L65922MH1989PLC052257: Expecting ':' delimiter: line 1 column 32768 (char 32767)
Error decoding JSON in row L55200MH1967PLC013837: Unterminated string starting at: line 1 column 32753 (char 32752)
Error decoding JSON in row L65191PN1994PLC076333: Unterminated string starting at: line 1 column 32767 (char 32766)
Error decoding JSON in row L27020MH1958PLC011238: Unterminated string starting at: line 1 column 32747 (char 32746)
Error decoding JSON in row L24299MH1977PLC019982: Unterminated string starting at: line 1 column 32767 (char 32766)
Error decoding JSON in row U11100GJ1989PLC032116: Expecting ':

In [30]:
file_new = "/content/drive/My Drive/kelp/task_output.xlsx"
CRtoLink_df.to_excel(file_new, index=False)